# Data from Transfermarkt

In [1]:
#https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
import pandas as pd
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_data_from_div_detailed(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the div with class 'responsive-table'
        div = soup.find('div', {'class': 'responsive-table'})
        
        if div:
            # Extract headers
            headers = div.find_all('th')
            header_texts = [header.get_text(strip=True) for header in headers]
            header_texts.append('Club')  # Adding 'Club' as a header
            
            # Extract contents
            rows = div.find_all('tr')
            for row in rows[1:]:  # Skipping the first row as it's the header row
                player_data = {}
                
                cells = row.find_all('td')
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                
                # Checking if the row is complete (same length as headers)
                if len(cell_texts) == len(header_texts) - 1:
                    for index, header in enumerate(header_texts[:-1]):
                        player_data[header] = cell_texts[index]
                    
                    # Extracting the club name from the 'title' attribute
                    club_tag = cells[3].find('a')
                    if club_tag and 'title' in club_tag.attrs:
                        club_name = club_tag['title']
                        player_data['Club'] = club_name
                    
                    data.append(player_data)
                
        else:
            print('Div not found.')
    else:
        print('Failed to fetch data.')
    
    return data

messi = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
ronaldo = "https://www.transfermarkt.com/cristiano-ronaldo/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
detailed_messi = get_data_from_div_detailed(messi)

detailed_ronaldo = get_data_from_div_detailed(ronaldo)

# Convert to DataFrame
df_messi_detailed = pd.DataFrame(detailed_messi)
df_ronaldo_detailed = pd.DataFrame(detailed_ronaldo)

print(df_messi_detailed)
#print(df_ronaldo)

   Season Competition reihenfolge ASC, saison_id DESC, verein_id ASC  \
0             Total :                                                  
1    2023                                                        MLS   
2    2023                                                Leagues Cup   
3   22/23                                                    Ligue 1   
4   22/23                                           Champions League   
..    ...         ...                                            ...   
65  05/06                                                     LaLiga   
66  05/06                                               Copa del Rey   
67  05/06                                           Champions League   
68  04/05                                                     LaLiga   
69  04/05                                           2ª B - Grupo III   

                   Club 1 - 15 16 - 30 31 - 45 45+ 46 - 60 61 - 75 76 - 90  \
0                           72     120     123   6     10

In [3]:
def get_data_from_div_trophies(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    data = []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all divs with class 'large-6 columns'
        divs = soup.find_all('div', {'class': 'large-6 columns'})
        
        for div in divs:
            # Extract headers within each 'large-6 columns' div
            headers = div.find_all('h2')
            
            for head in headers: 
                trophies_kind = re.search(r'>\s*([0-9]+x.*?)\s*<', str(head)) 
                if trophies_kind:
                    actual_trophies = trophies_kind.group(1)
                    
                    # Extract contents within each 'large-6 columns' div
                    rows = div.find_all('tr')
                    pat_2 = '[0-9]+'
                    
                    type_of_trophie_list = []
                    for row in rows[0:]:  # Skipping the first row as it's the header row
                        specific_trophie = re.search(pat_2, str(row)) 
                        if specific_trophie:
                            year = specific_trophie.group()
                            type_of_trophie_list.append(year)
                    
                    data.append((actual_trophies, type_of_trophie_list))
            
    else:
        print('Failed to fetch data.')
    
    return data

In [4]:
#messi = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
#result_messi = get_data_from_div_(messi)
#df_messi_detailed = pd.DataFrame(result_messi)
#print(df_messi_detailed)

messi = "https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/28003"
trophies_messi = get_data_from_div_trophies(messi)
df_trophies_messi_trophies = pd.DataFrame(trophies_messi)

#print(final_years)

In [5]:
def correct_the_seasons(df):
    final_years = []
    for years_lists in df[1].values:
        four_digits = []
        for year in years_lists:
            if len(year) == 4: 
                four_digits.append(year)
            else:
                #print(year)
                incremented_year = '{:02d}'.format(int(year) + 1)

                seasons = year+'/'+incremented_year
                four_digits.append(seasons)
        final_years.append(four_digits)
    
    return final_years
        
correted_seasons = correct_the_seasons(df_trophies_messi_trophies)
#print(correted_seasons)
df_trophies_messi_trophies[2] = correted_seasons
#print(df_trophies_messi_trophies)

In [6]:
def get_data_from_div_national(url):
    import requests
    from bs4 import BeautifulSoup
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    titles = []  # List to store all found titles
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all divs with class 'responsive-table'
        divs = soup.find_all('div', {'class': 'responsive-table'})
        
        # Skip the first div with the class 'responsive-table'
        for div in divs[1:]:
            if div:
                #print(div)
                span_elements = div.find_all('span', class_='icons_sprite')  # Using find_all to collect all span elements
                zentriert_elements = div.find_all('th', class_='zentriert')  # Using find_all to collect all z elements
                for element in zentriert_elements:
                    text = element.get_text(strip=True) 
                    titles.append(text)
                    #print(zentriert_elements)
                #Venue = div.find_all( class_='Venue')  # Using find_all to collect all span elements
                #print(Venue)
                
                count = 0
                for span_element in span_elements:
                    count+=1
                    title_text = span_element.get('title')
                    #print(count, title_text)
                    titles.append(title_text)  # Append each found title to the list
       
    
        for div in divs[1:]:
            if div:
                #print(div)
                td_elements = soup.select('tbody td.zentriert.hauptlink.no-border-rechts')
                #print(td_elements)
                # Extract the title from <a> tags within <td> elements
                special_lines = []
                for td in td_elements:
                    #print(td)
                    a_tag = td.find('a')
                    #print(a_tag)
                    if a_tag:
                        
                        reg = 'title="?[^"]+"?\/>'
                        special_line_1 = re.findall(reg, str(a_tag))
                        #print(special_line) 
                        
                        new_reg = '[^"]+'
                        special_line_2 = re.findall(new_reg, "".join(special_line_1))
                        #print(special_line_2[1])
                        
                        #special_line = a_tag.get('title')
                        #print(special_line_2)
                        special_lines.append(special_line_2[1])
                    
                desired_rows = div.find_all('tr')
                
                
                matches_list = []
                
                for div in desired_rows:
                    new_desired_row = div.find_all('td')
                    
                    match_details = []

                    for cell in new_desired_row:
                        text = cell.get_text(strip=True)
                        match_details.append(text if text else '0')

                    #matches_list.append(match_details)
                    #print(match_details, len(match_details))
                    
                    
                         #print(match_details[1]+match_details[3:4]+match_details[6])
                    if len(match_details) == 0: 
                        pass
                        
                    if len(match_details) == 2: 
                        type_of_matches = match_details+[0]*14
                        matches_list.append(type_of_matches)
                        
                    if len(match_details) == 11: 
                        not_played = [match_details[1]]+[match_details[3]]+[match_details[4]]+[match_details[6]]+match_details[8:11]+[0]*9
                        matches_list.append(not_played)
                    
                    if len(match_details) == 20: 
                        combined = [match_details[1]]+[match_details[3]]+[match_details[4]]+[match_details[6]]+ match_details[8:20]
                        matches_list.append(combined)
                    
                    

      
                
        else:
            print('Failed to fetch data.')
    
    #print(titles[1:5], titles[14:23])
    
    return matches_list

messi_national = "https://www.transfermarkt.com/lionel-messi/nationalmannschaft/spieler/28003/verein_id/3437/hauptwettbewerb//wettbewerb_id//start/2005-08-17/ende/2023-12-15/nurEinsatz/0/plus/1"
national_messi = get_data_from_div_national(messi_national)
National_column_names = ["Group", "Venue", "Date", "Home Team", "Away Team", "Result", "Pos", "Goal", "Assist", "Own Goal", "Yellow",
     "Double Yellow", "Red", "Substitutions on", "Substitutions off", "Minutes Played"]

# Create the pandas DataFrame 
national_messi_df = pd.DataFrame(national_messi, columns = National_column_names) 
 
#print(national_messi)  # This will print all the extracted titles as a list
#df_national_messi = pd.DataFrame(national_messi)

#print(df_national_messi)

Failed to fetch data.


In [7]:
for i in national_messi:
    print(len(i))

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16


In [8]:
column_names = ["Group", "Venue", "Date", "Home Team", "Away Team", "Result", "Pos", "Goal", "Assist", "Own Goal", "Yellow",
     "Double Yellow", "Red", "Substitutions on", "Substitutions off", "Minutes Played"]

# Create the pandas DataFrame 
df = pd.DataFrame(national_messi, columns = column_names) 
 
    
    
    
    
# Permanently changes the pandas settings
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
 
# All dataframes hereafter reflect these changes.
display(df)
 
#print('**RESET_OPTIONS**')
 
# Resets the options
#pd.reset_option('all')
#display(df)



,Group,Venue,Date,Home Team,Away Team,Result,Pos,Goal,Assist,Own Goal,Yellow,Double Yellow,Red,Substitutions on,Substitutions off,Minutes Played
0,0,International Friendlies,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,Budapest,"Aug 17, 2005",Hungary,Argentina,1:2,SS,0,0,0,0,0,65',64',0,1'
2,0,World Cup qualification South America,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Group A,Asunción,"Sep 3, 2005",Paraguay,Argentina,1:0,SS,0,0,0,0,0,0,80',0,10'
4,Group A,Buenos Aires,"Oct 9, 2005",Argentina,Peru,2:0,SS,0,1,0,0,0,0,0,0,90'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Group A,La Paz,"Sep 12, 2023",Bolivia,Argentina,0:3,Muscle injury,0,0,0,0,0,0,0,0,0
298,Group A,Buenos Aires,"Oct 13, 2023",Argentina,Paraguay,1:0,RW,0,0,0,0,0,0,53',0,37'
299,Group A,Lima,"Oct 18, 2023",Peru,Argentina,0:2,RW,2,0,0,0,0,0,0,0,90'
300,Group A,"La Boca, Buenos Aires, Capital Federal","Nov 17, 2023",Argentina,Uruguay,0:2,RW,0,0,0,0,0,0,0,0,90'


In [9]:
ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is B0FC-EBD7

 Directory of c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem1\adv_appl_stat\project\outplayed

27/12/2023  23.41    <DIR>          .
27/12/2023  23.41    <DIR>          ..
29/12/2023  16.53    <DIR>          .git
01/12/2023  11.17             5.183 .gitignore
27/12/2023  19.33    <DIR>          .vscode
08/12/2023  19.54           146.831 BallonDOr.csv
29/12/2023  16.50    <DIR>          data
29/12/2023  20.36            86.084 Get Data for Project.ipynb
29/12/2023  16.50    <DIR>          notebooks
27/12/2023  19.33               774 notes.txt
01/12/2023  11.17                82 README.md
27/12/2023  19.33                90 requirements.txt
               6 File(s)        239.044 bytes
               6 Dir(s)  332.237.021.184 bytes free


In [10]:
excel_file = pd.read_csv("data/BallonDOr_combined.csv")
print(excel_file)

      Year  Rank              Player                 Club   Nationality  \
0     1956     1    Stanley Matthews            Blackpool       England   
1     1956     2  Alfredo Di Stéfano       Real Madrid CF         Spain   
2     1956     3        Raymond Kopa       Real Madrid CF        France   
3     1956     4       Ferenc Puskás               Honvéd       Hungary   
4     1956     5          Lev Yashin        Dynamo Moskva  Soviet Union   
...    ...   ...                 ...                  ...           ...   
1895  2023    26       Jamal Musiala        Bayern Munich       Germany   
1896  2023    27      Nicolò Barella          Inter Milan         Italy   
1897  2023    28   Randal Kolo Muani  Eintracht Frankfurt        France   
1898  2023    28     Martin Ødegaard              Arsenal        Norway   
1899  2023    30          Rúben Dias      Manchester City      Portugal   

      Points   P1   P2   P3   P4   P5  Votes  RankPts   Share  Percent   Voted  
0       47.0  6.0 

In [11]:
#ballon_dor_df = pd.read_csv(r"BallonDOr.csv")
#print(ballon_dor_df["Year"]==2013)
excel_file = pd.read_csv("data/BallonDOr_combined.csv")
excel_file.loc[excel_file["Year"]>=2008]

,Year,Rank,Player,Club,Nationality,Points,P1,P2,P3,P4,P5,Votes,RankPts,Share,Percent,Voted
1498,2008,1,Cristiano Ronaldo,Manchester United,Portugal,446.0,77.0,11.0,4.0,1.0,3.0,96.0,50.0,0.3097,0.9292,1.0000
1499,2008,2,Lionel Messi,FC Barcelona,Argentina,281.0,6.0,33.0,27.0,14.0,10.0,90.0,25.0,0.1951,0.5854,0.9375
1500,2008,3,Fernando Torres,Liverpool,Spain,179.0,5.0,13.0,24.0,9.0,12.0,63.0,16.7,0.1243,0.3729,0.6563
1501,2008,4,Iker Casillas,Real Madrid CF,Spain,133.0,2.0,16.0,12.0,8.0,7.0,45.0,12.5,0.0924,0.2771,0.4688
1502,2008,5,Xavi,FC Barcelona,Spain,97.0,3.0,9.0,4.0,15.0,4.0,35.0,10.0,0.0674,0.2021,0.3646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,2023,26,Jamal Musiala,Bayern Munich,Germany,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN
1896,2023,27,Nicolò Barella,Inter Milan,Italy,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN
1897,2023,28,Randal Kolo Muani,Eintracht Frankfurt,France,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN
1898,2023,28,Martin Ødegaard,Arsenal,Norway,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN


In [12]:
ronaldo = "https://www.transfermarkt.com/cristiano-ronaldo/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"

In [13]:
#print(ballon_dor_df["Year"])
ballon_2008_2018 = excel_file.loc[excel_file["Year"]>=2008]

p_08_18 = ballon_2008_2018["Player"].unique()
print(p_08_18)

['Cristiano Ronaldo' 'Lionel Messi' 'Fernando Torres' 'Iker Casillas'
 'Xavi' 'Andrei Arshavin' 'David Villa' 'Kaká' 'Zlatan Ibrahimović'
 'Steven Gerrard' 'Marcos Senna' 'Emmanuel Adebayor' 'Wayne Rooney'
 'Sergio Agüero' 'Frank Lampard' 'Franck Ribéry' "Samuel Eto'o"
 'Gianluigi Buffon' 'Cesc Fàbregas' 'Michael Ballack' 'Sergio Ramos'
 'Nemanja Vidić' 'Didier Drogba' 'Edwin van der Sar' 'Ruud van Nistelrooy'
 'Andrés Iniesta' 'Edin Džeko' 'Ryan Giggs' 'Thierry Henry'
 'Luís Fabiano Clemente' 'Diego Forlán' 'Yoann Gourcuff'
 'Júlio César Soares Espíndola' 'Maicon' 'Diego Ribas da Cunha'
 'John Terry' 'Yaya Touré' 'Wesley Sneijder' 'Xabi Alonso' 'Carles Puyol'
 'Mesut Özil' 'Arjen Robben' 'Thomas Müller' 'Bastian Schweinsteiger'
 'Asamoah Gyan' 'Júlio César' 'Miroslav Klose' 'Philipp Lahm' 'Dani Alves'
 'Luis Suárez Díaz' 'Neymar' 'Éric Abidal' 'Karim Benzema' 'Nani'
 'Gerard Piqué' 'Radamel Falcao' 'Andrea Pirlo' 'Robin van Persie'
 'Manuel Neuer' 'Sergio Busquets' 'Mario Balotelli' '

In [14]:
print(p_08_18[8])

Zlatan Ibrahimović


In [15]:
print(p_08_18[8].lower())

zlatan ibrahimović


In [16]:
import unicodedata

lower_list = []

for i in p_08_18:
    #j = i.split()
    #q = "-".join(j)
    j = i.lower()
    
    slug =  unicodedata.normalize('NFD', j).encode('ascii', 'ignore').decode('ascii')
    #print(slug)
    lower_list.append(slug)
    
    
print(lower_list)

['cristiano ronaldo', 'lionel messi', 'fernando torres', 'iker casillas', 'xavi', 'andrei arshavin', 'david villa', 'kaka', 'zlatan ibrahimovic', 'steven gerrard', 'marcos senna', 'emmanuel adebayor', 'wayne rooney', 'sergio aguero', 'frank lampard', 'franck ribery', "samuel eto'o", 'gianluigi buffon', 'cesc fabregas', 'michael ballack', 'sergio ramos', 'nemanja vidic', 'didier drogba', 'edwin van der sar', 'ruud van nistelrooy', 'andres iniesta', 'edin dzeko', 'ryan giggs', 'thierry henry', 'luis fabiano clemente', 'diego forlan', 'yoann gourcuff', 'julio cesar soares espindola', 'maicon', 'diego ribas da cunha', 'john terry', 'yaya toure', 'wesley sneijder', 'xabi alonso', 'carles puyol', 'mesut ozil', 'arjen robben', 'thomas muller', 'bastian schweinsteiger', 'asamoah gyan', 'julio cesar', 'miroslav klose', 'philipp lahm', 'dani alves', 'luis suarez diaz', 'neymar', 'eric abidal', 'karim benzema', 'nani', 'gerard pique', 'radamel falcao', 'andrea pirlo', 'robin van persie', 'manuel 

In [17]:
print(lower_list[8])

zlatan ibrahimovic


In [18]:
#Player name, player_id
#Manually scraped becuase it takes more than 180h+ for jupyter notebook/code and risk of memory error if interent fails 
my_dict = {"cristiano ronaldo": 8198, "lionel messi": 28003, "fernando torres": 7767, "iker casillas": 3979,"xavi": 7607, 
           "andrei arshavin":15378, "david villa": 7980, "kaka": 3366, "zlatan ibrahimovic": 3455, "steven gerrard":3109,
          "marcos senna": 7840, "emmanuel adebayor":8883, "wayne rooney": 3332, "sergio aguero": 26399,
           "frank lampard": 3163,"franck ribery":22068, "samuel eto'o": 4257, "gianluigi buffon": 5023, 
           "cesc fabregas":8806, "michael ballack": 63, "sergio ramos": 25557, "nemanja vidic": 19726, 
           "didier drogba": 3924, "edwin van der sar": 3516, "ruud van nistelrooy": 3407,"andres iniesta": 7600, 
           "edin dzeko":28396, "ryan giggs": 3406, "thierry henry": 3207, "luis fabiano clemente": 19082,
           "diego forlan": 3408, "yoann gourcuff":18912, "julio cesar soares espindola": 22412, "maicon": 18301,
           "diego ribas da cunha": 4248, "john terry": 3160, "yaya toure": 13091,"wesley sneijder": 4673, 
           "xabi alonso":7476, "carles puyol": 7594, "mesut ozil": 35664, "arjen robben":4360, "thomas muller": 58358,
          "bastian schweinsteiger": 2514, "asamoah gyan": 26802,"julio cesar": 22412, "miroslav klose": 10,
           "philipp lahm": 2219, "dani alves": 15951,"luis suarez diaz":44352,"neymar": 68290, "eric abidal": 5283, 
          "karim benzema": 18922, "nani": 33706, "gerard pique": 18944, "radamel falcao": 39152, "andrea pirlo": 5817,
          "robin van persie": 4380,"manuel neuer": 17259, "sergio busquets": 65230, "mario balotelli":45146, 
          "gareth bale": 39381, "robert lewandowski": 38253, "edinson cavani": 48280, "thiago silva":29241, 
          "eden hazard": 50202, "james rodriguez": 88103, "toni kroos": 31909, "angel di maria": 45320, 
           "diego costa": 44779, "mario gotze": 74842, "javier mascherano": 19981,"thibaut courtois": 108390, 
           "paul pogba": 122153, "alexis sanchez":40433, "arturo vidal": 37666, "kevin de bruyne": 88755,
           "ivan rakitic": 32467, "antoine griezmann": 125781, "riyad mahrez": 171424, "jamie vardy":197838,
           "pepe": 14132, "pierre-emerick aubameyang": 58864, "rui patricio":45026, "luka modric": 27992, 
           "dimitri payet":37647, "paulo dybala":206050, "diego godin": 54928, "gonzalo higuain":39153,"koke": 74229,
           "hugo lloris": 17965, "kylian mbappe": 342229, "n'golo kante":225083, "harry kane":132098,"isco":85288,
           "marcelo": 44501, "david de gea": 59377, "leonardo bonucci": 39983, "sadio mane": 200512, "jan oblak": 121483,
           "mats hummels": 39728, "philippe coutinho": 80444, "dries mertens": 56416, "mohamed salah":148455, 
           "raphael varane": 164770, "roberto firmino":131789, "alisson": 105470, "mario mandzukic":34572, 
           "virgil van dijk": 139208, "bernardo silva": 241641,"frenkie de jong":326330, "raheem sterling":134425, 
           "matthijs de ligt": 326031, "trent alexander-arnold": 314353, "dusan tadic": 36139, "son heung-min":91845,
           "kalidou koulibaly": 93128, "marc-andre ter stegen": 74857, "georginio wijnaldum": 49499, "joao felix":462250,
           "marquinhos": 181767, "donny van de beek": 288255, "jorginho": 102017, "gianluigi donnarumma":315858,
           "erling haaland": 418560, "romelu lukaku":96341, "giorgio chiellini": 29260, "luis suarez":44352, 
           "simon kjr":48859, "simon kjaer": 48859, "mason mount": 346483, "bruno fernandes": 240306, 
           "lautaro martinez":406625, "pedri": 683840, "phil foden": 406635, "gerard moreno": 177467, 
           "nicolo barella": 255942, "ruben dias": 258004, "cesar azpilicueta": 57500, "vinicius junior": 371998,
           "sebastien haller":181375, "fabinho": 225693, "rafael leao": 357164, "casemiro": 16306,
           "luis diaz": 480692, "dusan vlahovic": 357498, "joao cancelo": 182712, "joshua kimmich": 161056,
           "mike maignan": 182906, "christopher nkunku": 344381, "darwin nunez": 546543, "antonio rudiger": 86202,
           "rodri": 357565, "julian alvarez": 576024, "victor osimhen": 401923, "yassine bounou":207834, "buno": 207834,
           "ilkay gundogan":53622, "emiliano martinez":111873, "khvicha kvaratskhelia": 502670, "jude bellingham":581678,
           "kim min-jae":503482, "kim min jae":503482, "andre onana": 234509, "bukayo saka":433177, 
           "josko gvardiol":475959, "josko gvardiol":475959, "jamal musiala":580195, "randal kolo muani":487969,
           "martin degaard": 316264, "martin-odegaard":316264}

##Took 1h and 30 min

In [74]:
players = list(my_dict.keys())
index = 4
print(players[index])
print(f'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/{my_dict[players[index]]}/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=WM14&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0')
print(f'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/{my_dict[players[index]]}/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=WM14')

xavi
https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/7607/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=WM14&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0
https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/7607/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=WM14


In [ ]:
# second dict for second ids :D
second_dict = {'cristiano ronaldo': 3300, 'lionel messi': 3437, 'fernando torres': 3375, 'iker casillas': 3375,
               'xavi': }

### Getting tournament data

In [19]:
def get_data_from_div_tournament(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the divs with class 'box'
        divs = soup.find_all('div', {'class': 'box'})
        
        if divs:
            for div in divs:
                # only work with tables that have headers
                try:
                    table_header = div.find('h2', {'class': 'content-box-headline'}).get_text(strip=True)
                except:
                    continue
                
                # only get International cups data
                if table_header != 'International cups':
                    continue

                headers = div.find_all('th')
                header_texts = [header.get_text(strip=True) for header in headers]
                
                # Extract contents
                rows = div.find_all('tr')
                for row in rows[2:]: # Skipping the first row as it's the header row
                    player_data = {}
                    
                    cells = row.find_all('td')
                    cell_texts = [cell.get_text(strip=True) for cell in cells]
                    del cell_texts[1]

                    # different stats depending on if goalkeeper or not
                    if len(header_texts) == 12:
                        # goalkeeper
                        headers = ['Season', 'Competition', 'Club', 'Appearances',
                                       'Goals', 'Yellow cards', 'Second yellow cards', 'Red cards',
                                       'Goals conceded', 'Clean sheets', 'Minutes played']
                    
                    elif len(header_texts) == 11:
                        # all other players
                        headers = ['Season', 'Competition', 'Club', 'Appearances', 'Goals', 'Assists',
                                   'Yellow cards', 'Sesond yellow cards', 'Red cards', 'Minutes played']
                        
                    for index, header in enumerate(headers):
                        # getting club name from icon attribute
                        if header == 'Club':
                            club_tag = cells[3].find('a')
                            club_name = club_tag['title']
                            player_data[header] = club_name

                        # ensuring raw number for minutes played
                        elif header == 'Minutes played':
                            player_data[header] = cell_texts[index].replace('.', '').replace('\'', '')

                        else:    
                            player_data[header] = cell_texts[index]
                    
                    data.append(player_data)
                
        else:
            print('Div not found.')
    else:
        print('Failed to fetch data.')
    
    return data

### Getting big cup data (World Cup and Euro Cup)

In [53]:
def get_data_from_div_big_cup(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all 'box' classes
        boxes = soup.find_all('div', {'class': 'box'})

        if boxes:
            for box in boxes:
                # find header of box
                try:
                    header = box.find('h2', {'class': 'content-box-headline'}).get_text(strip=True)
                except:
                    continue
                
                # ignore all uninteresting tables
                if header != 'Detailed stats':
                    continue

                # narrow down to table class
                table = box.find('div', {'class': 'responsive-table'})

                # grab all the column names
                cols = []
                header = table.find('thead')
                header_cells = header.find_all('th')
                for header_cell in header_cells:
                    header_text = header_cell.get_text(strip=True)
                    if header_text == '':
                        # need to get context from icon
                        icon = header_cell.find('span')
                        if icon == None: # first col is always None
                            continue
                        
                        cols.append(icon['title'])
                    else:
                        cols.append(header_text)
                del cols[2] # venue - not interested

                # extract contents
                rows = table.find_all('tr')
                for row in rows[2:-1]: # first two and last rows are bad
                    player_data = {}
                    # get all cells
                    cells = row.find_all('td')
                    cell_texts = [cell.get_text(strip=True) for cell in cells]

                    # if player was on the bench, just ignore the row
                    if 'on the bench' in cell_texts[-1]:
                        continue

                    del cell_texts[0] # useless
                    del cell_texts[2] # venue - not interested
                    del cell_texts[3] # flag of team - not interested
                    del cell_texts[4] # flag of other team - not interested

                    # format minutes played
                    cell_texts[-1] = int(cell_texts[-1].replace('\'', '').replace('.', ''))

                    for index, header in enumerate(cols):
                        player_data[header] = cell_texts[index]

                    # get club data
                    club = cells[2].find('a')['title']
                    player_data['Club'] = club
                    
                    data.append(player_data)
                
        else:
            print('Div not found.')
    else:
        print('Failed to fetch data.')

    tmp_df = pd.DataFrame(data)
    data_new = {
        'Matchday': tmp_df['Matchday'][0],
        'Club': tmp_df['Club'][0],
        'Year': tmp_df['Date'][0][-2:],
        }
    # summing other stats
    for col in cols[7:]:
        # special case with cards as it is the minute of card given so just count it
        if 'card' in col:
            data_new[col] = tmp_df[col].count()
        else:
            data_new[col] = tmp_df[col].sum()
    
    
    return data_new

In [21]:
world_cup_years = ['10', '14', '18', '22']
euro_years = ['08', '12', '16', '20']
euro_url = f'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/{player_id}/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=EM{year}&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0'
world_cup_url = f'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/{player_id}/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=WM{year}&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0'

NameError: name 'player_id' is not defined

In [52]:
ronaldo_WC_url = 'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/8198/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=WM14&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0'
ronaldo_EU_url = 'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/8198/verein_id/3300/plus/1?hauptwettbewerb=&wettbewerb_id=EM08&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0'
Iker_WC_url = 'https://www.transfermarkt.co.uk/iker-casillas/nationalmannschaft/spieler/3979/verein_id/3375/plus/1?hauptwettbewerb=&wettbewerb_id=WM14&trainer_id=&start=Jun+3%2C+2000&ende=Jun+27%2C+2016&nurEinsatz=0'
print(get_data_from_div_big_cup(ronaldo_WC_url))
print(get_data_from_div_big_cup(ronaldo_EU_url))
print(get_data_from_div_big_cup(Iker_WC_url))

{'Matchday': 'Group G', 'Club': 'Real Madrid', 'Year': '14', 'Goals': '1', 'Assists': '1', 'Own goals': '', 'Yellow cards': 3, 'Second yellow cards': 3, 'Red cards': 3, 'Substitutions on': '', 'Substitutions off': '', 'Minutes played': 270}
{'Matchday': 'Group A', 'Club': 'Manchester United', 'Year': '08', 'Goals': '1', 'Assists': '21', 'Own goals': '', 'Yellow cards': 3, 'Second yellow cards': 3, 'Red cards': 3, 'Substitutions on': '', 'Substitutions off': '', 'Minutes played': 270}
{'Matchday': 'Group B', 'Club': 'Real Madrid Castilla', 'Year': '14', 'Goals': '', 'Assists': '', 'Own goals': '', 'Yellow cards': 2, 'Second yellow cards': 2, 'Red cards': 2, 'Substitutions on': '', 'Substitutions off': '', 'Minutes played': 180}


In [ ]:
### All the following strings are formatted with player_id in the actual code 
# Details_stats = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
    #Season, Competition Club, 15min intervals of scored goals, Ekstra time, Total amount of goals 
    
    
# Trophies = https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/34572
    # Kind of trophies, which year/season the specific kind of trophy was won in.


# National team = https://www.transfermarkt.com/mario-mandzukic/nationalmannschaft/spieler/34572/verein_id/3556/hauptwettbewerb//wettbewerb_id//start/2007-11-17/ende/2018-07-15/nurEinsatz/0/plus/1
    #Kind of match in which tournament, Club, Opponant, Date, Result, Goals, Assists, Own goals, 
    # Minute for what kind of card was given to the player, minutes played of the match

National_column_names = ["Group", "Venue", "Date", "Home Team", "Away Team", "Result", "Pos", "Goal", "Assist", "Own Goal", "Yellow",
     "Double Yellow", "Red", "Substitutions on", "Substitutions off", "Minutes Played"]

# Create the pandas DataFrame 
national_messi_df = pd.DataFrame(national_messi, columns = National_column_names)     
    
    
# Stats per tournament = https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/34572
    #Competition, Club name, appearances for the club, Goals, asissists, cards, Minutes Played
    #The aboed are mentioned for each type of legue (National legue, Domestic cups and International cups)

# Stats per club = https://www.transfermarkt.com/mario-mandzukic/leistungsdatenverein/spieler/34572
    #Club name, appearances for the club, Goals, asissists, cards, Minutes Played, 
    #The above are mentioned for each league the player played in 


players = []
tournament_dfs = []
for year in range(2008,2024): 
    #print(year)
    m = ballon_2008_2018.loc[ballon_2008_2018["Year"]== year]
    
    for i in m["Player"].values: 
        slug =  unicodedata.normalize('NFD', i).encode('ascii', 'ignore').decode('ascii')
        lowered = slug.lower()
        try: 
            if lowered not in players:
                players.append(lowered)
                player_id = my_dict[lowered]
                print(lowered, player_id)
                Detailed_url = f"https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
                Trophies_url = f"https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/{player_id}"
                
                National_url = f"https://www.transfermarkt.com/mario-mandzukic/nationalmannschaft/spieler/{player_id}/verein_id/3556/hauptwettbewerb//wettbewerb_id//start/2007-11-17/ende/2018-07-15/nurEinsatz/0/plus/1"
                Stats_per_tournament_url = f"https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/{player_id}"
                Stats_per_club = f"https://www.transfermarkt.com/mario-mandzukic/leistungsdatenverein/spieler/{player_id}"
                
                
                ### The detailed df
                # Detailed = get_data_from_div_detailed(Detailed_url)
                # df_detailed = pd.DataFrame(Detailed)

                ### The trophies
                # trophies_df = get_data_from_div_trophies(Trophies_url)
                # df_trophies = pd.DataFrame(trophies_df)
                
                # correted_seasons = correct_the_seasons(df_trophies)
                # df_trophies[2] = correted_seasons
                #print(df_trophies)
                
                ### National Teams
                # national_stats = get_data_from_div_national(National_url)
                # national_stats_df = pd.DataFrame(national_stats, columns = National_column_names)  
                
                #national_messi = get_data_from_div_national(National_url)
                
                ### Tournament Data
                # tournament_stats = get_data_from_div_tournament(Stats_per_tournament_url)
                # tournament_stats_df = pd.DataFrame(tournament_stats)
                # tournament_stats_df['Player'] = i
                # tournament_dfs.append(tournament_stats_df)

                ### Euro and World Cup Data
                world_cup_years = ['10', '14', '18', '22']
                euro_years = ['08', '12', '16', '20']
                world_cup_dfs = []
                for year in world_cup_years:
                    WC_url = f'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/{player_id}/verein_id/{second_dict[player_id]}/plus/1?hauptwettbewerb=&wettbewerb_id=WM{year}&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0'
                    world_cup_df = pd.DataFrame(get_data_from_div_big_cup(WC_url))
                    world_cup_df['Player'] = i
                    world_cup_dfs.append(world_cup_df)
                euro_dfs = []
                for year in euro_years:
                    euro_url = f'https://www.transfermarkt.co.uk/cristiano-ronaldo/nationalmannschaft/spieler/{player_id}/verein_id/{second_dict[player_id]}/plus/1?hauptwettbewerb=&wettbewerb_id=EM{year}&trainer_id=&start=Aug+20%2C+2003&ende=Dec+29%2C+2023&nurEinsatz=0'
                    euro_df = pd.DataFrame(get_data_from_div_big_cup(euro_url))
                    euro_df['Player'] = i
                    euro_dfs.append(euro_df)


                
                #players.append(df_player)
                # break
                
        except: 
            print("Player is missing\n")
       # break
        #result_zlatan = get_data_from_div(zlatan)

        # Convert to DataFrame
        #df_zlatan = pd.DataFrame(result_zlatan)
        
        

In [ ]:
# combining and saving tournament stats
tournament_stats_combined = pd.concat(tournament_dfs)
tournament_stats_combined.to_csv('data/tournament_stats_new.csv', index=False)

In [ ]:
# combining and saving world cup + euro stats
WC_combined = pd.concat(world_cup_dfs)
WC_combined.to_csv('data/world_cups.csv', index=False)

euro_combined = pd.concat(euro_dfs)
euro_combined.to_csv('data/euros.csv', index=False)

In [ ]:
error

In [ ]:
my_dict

In [ ]:
index = 0
player_keys = list(my_dict.keys())
player = player_keys[index]
print(player)
print(f'https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/{my_dict[player]}')

In [ ]:
print(f'https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/{45146}')

In [ ]:
player_id = 3979 # Iker
# player_id = 8198 # Cristiano
# player_id = 139208 # virgil
player_id = 3169
player_id = 5023 # frank lampart
url = f"https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/{player_id}"
print(url)
get_data_from_div_tournament(url)

In [ ]:
for i, j in enumerate(lower_list): 
    f'df_{i}' = j
    print(df_i)

In [ ]:
print(df_neymar)